# Binary classification Model

### train / test split

In [180]:
import numpy as np
import torch
from torch import nn
from sympy import false

# Load all the test and train data as numpy objects.
X_train = np.loadtxt("../../datasets/merged/X_train_imputed_scaled.csv", delimiter=",", skiprows=1)
X_test = np.loadtxt("../../datasets/merged/X_test_imputed_scaled.csv", delimiter=",", skiprows=1)
y_train = np.loadtxt("../../datasets/merged/y_train.csv", delimiter=",", skiprows=1)
y_test = np.loadtxt("../../datasets/merged/y_test.csv", delimiter=",", skiprows=1)

# Delete the id column.
X_train = np.delete(X_train, 0, axis=1)
X_test = np.delete(X_test, 0, axis=1)
y_train = np.delete(y_train, 0, axis=1)
y_test = np.delete(y_test, 0, axis=1)

# Change into tensor.
X_train = torch.from_numpy(X_train).type(torch.float32).squeeze()
X_test = torch.from_numpy(X_test).type(torch.float32).squeeze()
y_train = torch.from_numpy(y_train).type(torch.float32).squeeze()
y_test = torch.from_numpy(y_test).type(torch.float32).squeeze()


In [181]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([10018, 7]),
 torch.Size([2648, 7]),
 torch.Size([10018]),
 torch.Size([2648]))

In [182]:
type(X_train), X_train.dtype

(torch.Tensor, torch.float32)

In [183]:
type(y_train), y_train.dtype

(torch.Tensor, torch.float32)

In [184]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [185]:
torch.cuda.is_available()

True

### Construct Model Class

In [186]:
class ExoplanetsV0(nn.Module):

    def __init__(self, input_features, output_features, hidden_units=8):
        super().__init__()

        self.input_features = input_features
        self.output_features = output_features
        self.hidden_units = hidden_units

        self.layers = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=int(hidden_units/2)),
            nn.ReLU(),
            nn.Linear(in_features=int(hidden_units/2), out_features=int(hidden_units / 4)),
            nn.ReLU(),
            nn.Linear(in_features=int(hidden_units/4), out_features=output_features),
        )

    def forward(self, x):
        return self.layers(x)


# Create a test model
test = ExoplanetsV0(input_features=X_test.shape[1], output_features=1, hidden_units=32).to(device)
print(test, test.input_features, test.output_features, test.hidden_units)
del test

ExoplanetsV0(
  (layers): Sequential(
    (0): Linear(in_features=7, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=1, bias=True)
  )
) 7 1 32


In [187]:
X_train.shape, y_train.shape

(torch.Size([10018, 7]), torch.Size([10018]))

### Accuracy function

In [188]:
# Calculate accuracy
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

# Train Model

Model outputs are raw **Logits**

We are converting logits into prediction probabilities by passing them to some kind of activation function (int this case `nn.Sigmoid()`) Then we can convert our model's prediction probabilities to **prediction labels** by taking the `argmaX_scaled()`

### Plot analysis

In [189]:
import matplotlib.pyplot as plt

def makePlots(track_accuracy, track_loss, track_f1, track_precision, track_recall, track_auc,
              track_test_accuracy, track_test_loss, track_test_f1, track_test_precision,
              track_test_recall, track_test_auc):
    epoch_range = range(1, len(track_accuracy) + 1)

    plt.figure(figsize=(18, 12))

    # Accuracy
    plt.subplot(3, 2, 1)
    plt.plot(epoch_range, track_accuracy, label='Train Accuracy')
    plt.plot(epoch_range, track_test_accuracy, label='Test Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)

    # Loss
    plt.subplot(3, 2, 2)
    plt.plot(epoch_range, track_loss, label='Train Loss')
    plt.plot(epoch_range, track_test_loss, label='Test Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # F1 Score
    plt.subplot(3, 2, 3)
    plt.plot(epoch_range, track_f1, label='Train F1')
    plt.plot(epoch_range, track_test_f1, label='Test F1')
    plt.title('F1 Score')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.grid(True)

    # Precision
    plt.subplot(3, 2, 4)
    plt.plot(epoch_range, track_precision, label='Train Precision')
    plt.plot(epoch_range, track_test_precision, label='Test Precision')
    plt.title('Precision')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.legend()
    plt.grid(True)

    # Recall
    plt.subplot(3, 2, 5)
    plt.plot(epoch_range, track_recall, label='Train Recall')
    plt.plot(epoch_range, track_test_recall, label='Test Recall')
    plt.title('Recall')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.legend()
    plt.grid(True)

    # AUC
    plt.subplot(3, 2, 6)
    plt.plot(epoch_range, track_auc, label='Train AUC')
    plt.plot(epoch_range, track_test_auc, label='Test AUC')
    plt.title('AUC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('training_metrics.png', dpi=300)  # Zapisuje wykres w wysokiej jakości
    plt.show()


### Model training

In [190]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

def trainModel(model, X_train, y_train, X_test, y_test, epochs, lr, threshold, isCrossValidation=False):

    # Setting seed for reproducibility.
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)

    model.to(device)

    # Epsilon for float comparison.
    EPS = 0.00001

    # Tracking data for the plots.
    track_accuracy = []
    track_loss = []
    track_f1 = []
    track_precision = []
    track_recall = []
    track_auc = []

    track_test_f1 = []
    track_test_precision = []
    track_test_recall = []
    track_test_auc = []
    track_test_accuracy = []
    track_test_loss = []


    # Put data to the target device.
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    # Setting weights to reduce data imbalance.
    weight_for_0 = len(y_test) + len(y_train) / (2 * ((y_test < EPS).sum() + (y_train < EPS).sum()))
    weight_for_1 = len(y_test) + len(y_train) / (2 * ((y_test - 1 < EPS).sum() + (y_train - 1 < EPS).sum()))
    pos_weight = torch.tensor([weight_for_1 / weight_for_0]).to(device)

    loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)


    for epoch in range(epochs):
        ### Training
        model.train()

        # 1. Forward pass
        y_logits = model(X_train).squeeze() # Raw logits.
        y_probs = torch.sigmoid(y_logits)   # Probability.
        y_pred = (y_probs  >= threshold).int().to(device) # Prediction

        # 2. Calculate the loss
        loss = loss_fn(y_logits, y_train)
        track_loss.append(loss.item())

        acc = accuracy_fn(y_true=y_train.detach(), y_pred=y_pred.detach())
        precision = precision_score(y_true=y_train.detach().cpu(), y_pred=y_pred.detach().cpu(), zero_division=0) * 100
        recall = recall_score(y_true=y_train.detach().cpu(), y_pred=y_pred.detach().cpu(), zero_division=0) * 100
        f1 = f1_score(y_true=y_train.detach().cpu(), y_pred=y_pred.detach().cpu(), zero_division=0) * 100
        auc = roc_auc_score(y_true=y_train.detach().cpu(), y_score=y_probs.detach().cpu())

        track_accuracy.append(acc)
        track_precision.append(precision)
        track_recall.append(recall)
        track_f1.append(f1)
        track_auc.append(auc)

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Backpropagation algorithm
        loss.backward()

        # 5. Gradient descent algorithm
        optimizer.step()

        ### Testing
        model.eval()
        with torch.inference_mode():
            # 1. Forward pass
            test_logits = model(X_test).squeeze()                  # logits
            test_probs = torch.sigmoid(test_logits).to(device)     # Probability
            test_pred = (test_probs  >= threshold).int().to(device)         # 1 or 0

            # 2. Calculate test loss/acc

            test_loss = loss_fn(test_logits, y_test)
            track_test_loss.append(test_loss.item())

            test_acc = accuracy_fn(y_true=y_test, y_pred=test_pred)
            test_precision = precision_score(y_true=y_test.cpu(), y_pred=test_pred.cpu(), zero_division=0) * 100
            test_recall = recall_score(y_true=y_test.cpu(), y_pred=test_pred.cpu(), zero_division=0) * 100
            test_f1 = f1_score(y_true=y_test.cpu(), y_pred=test_pred.cpu(), zero_division=0) * 100
            test_auc = roc_auc_score(y_true=y_test.cpu(), y_score=test_probs.cpu())

            track_test_accuracy.append(test_acc)
            track_test_precision.append(test_precision)
            track_test_recall.append(test_recall)
            track_test_f1.append(test_f1)
            track_test_auc.append(test_auc)

        # Print out data.
        if isCrossValidation and (epoch % int(epochs / 10) == 0 or epoch == epochs - 1):
            print(
                f"Epoch {epoch} \n"
                f" loss: {loss:.5f} | accuracy: {acc:.2f}% | Precision: {precision:.2f}% | Recall: {recall:.2f}% | f1: {f1:.2f}% | auc: {auc:.2f} \n"
                f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}% | Test precision: {test_precision:.2f}% | Test recall: {test_recall:.2f}% | Test f1: {test_f1:.2f}% | Test auc: {test_auc:.2f} \n")

    # If it is the test, not validation.
    if not isCrossValidation:
        makePlots(track_accuracy, track_loss, track_f1, track_precision, track_recall, track_auc,
                  track_test_accuracy, track_test_loss, track_test_f1, track_test_precision,
                  track_test_recall, track_test_auc)
    return test_acc


# Cross-validation

In [191]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True)
cv_scores = []
for train, test in kfold.split(X_train, y_train):
    # create model, train, and get accuracy
    model = ExoplanetsV0(input_features=X_test.shape[1], output_features=1, hidden_units=32).to(device)
    acc = trainModel(model, X_train[train], y_train[train], X_train[test], y_train[test], 2500, 0.01, 0.5, True)
    print("Accuracy: %.2f" % acc)
    cv_scores.append(acc)

# evaluate the model
model_acc = np.mean(cv_scores)
model_std = np.std(cv_scores)
print(f"Stats (cross validation): {model_acc:.2f}% (+/- {model_std:.2f}%)")

model_1 = ExoplanetsV0(input_features=X_test.shape[1], output_features=1, hidden_units=32).to(device)
trainModel(model_1, X_train, y_train, X_test, y_test, 2500, 0.01, 0.5, false)

Epoch 0 
 loss: 0.70558 | accuracy: 51.46% | Precision: 0.00% | Recall: 0.00% | f1: 0.00% | auc: 0.42 
Test loss: 0.70517 | Test accuracy: 51.50% | Test precision: 0.00% | Test recall: 0.00% | Test f1: 0.00% | Test auc: 0.41 

Epoch 250 
 loss: 0.68800 | accuracy: 51.63% | Precision: 0.00% | Recall: 0.00% | f1: 0.00% | auc: 0.72 
Test loss: 0.68809 | Test accuracy: 51.65% | Test precision: 0.00% | Test recall: 0.00% | Test f1: 0.00% | Test auc: 0.72 

Epoch 500 
 loss: 0.67356 | accuracy: 51.85% | Precision: 81.48% | Recall: 0.57% | f1: 1.13% | auc: 0.76 
Test loss: 0.67416 | Test accuracy: 51.75% | Test precision: 75.00% | Test recall: 0.31% | Test f1: 0.62% | Test auc: 0.75 

Epoch 750 
 loss: 0.65519 | accuracy: 65.06% | Precision: 69.11% | Recall: 50.21% | f1: 58.16% | auc: 0.75 
Test loss: 0.65623 | Test accuracy: 64.72% | Test precision: 68.77% | Test recall: 49.54% | Test f1: 57.59% | Test auc: 0.75 



KeyboardInterrupt: 

# Saving and loading already trained Model

Creating template / method for effective saving already trained model, so user don't have to wait for it to finish training before usage.

In [403]:
from pathlib import Path

MODEL_PATH = Path("../../saved-models/nn")
MODEL_PATH.mkdir(parents = True, exist_ok = True)

def SaveModel(model, model_name: str):
    # 1. Create models directory if it doesn't exist yet

    # 2. Create model save path
    MODEL_NAME = model_name + ".pth"
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

    # 3. Save the model state dict
    save = [model.state_dict(), model.input_features, model.output_features, model.hidden_units]
    print(f"Saving model to: {MODEL_SAVE_PATH}")
    torch.save(obj = save,
               f = MODEL_SAVE_PATH)

SaveModel(model_1, "nn_exoplanets0")

Saving model to: ../../saved-models/nn/nn_exoplanets0.pth


In [386]:
!ls -l ../../saved-models

total 4
drwxrwxr-x 2 hubert hubert 4096 Oct  5 00:23 nn


### Loading Trained model



In [425]:
# To load in a saved state_dict we have to instantiate a new instace of our model class
def load_model(model_name):
    model_name += ".pth"
    load = torch.load(f = MODEL_PATH / model_name)
    loaded_model = ExoplanetsV0(input_features=load[1], output_features=load[2], hidden_units=load[3])
    loaded_model.load_state_dict(load[0])
    return loaded_model

# Using already trained model to evaluate new input data

In [423]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

def load_single_csv(folder_path= "../../input_data"):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

    if len(csv_files) == 0:
        raise FileNotFoundError(f" Brak pliku CSV w folderze: {folder_path}")
    elif len(csv_files) > 1:
        raise ValueError(f"W folderze {folder_path} znaleziono więcej niż jeden plik CSV: {csv_files}")

    file_path = os.path.join(folder_path, csv_files[0])
    print(f" Wczytuję plik: {file_path}")

    data = np.loadtxt(file_path, delimiter=",", skiprows=1)
    data = np.delete(data, 0, axis = 1)
    data = torch.from_numpy(data).type(torch.float32).squeeze()
    return data

def use_model(model_name):
    # Loading already trained model from "*.pth" file
    loaded_model = load_model(model_name)
    loaded_model.eval()

    # Input is always in the same place
    data = load_single_csv()
    with torch.inference_mode():
        logits = loaded_model(data)
        probabilities = torch.sigmoid(logits).squeeze()
        predictions = (probabilities > 0.5).int()

        # Saving results to csv format
        results = pd.DataFrame({
        "probability": probabilities.cpu().numpy(),
        "prediction": predictions.cpu().numpy()
        })

        output_path = "predictions.csv"
        results.to_csv(output_path, index=False)

        print(f" Zapisano wyniki do pliku: {output_path}")
        print(results.head())

    # Deleting csv so that there is only one file there at a time

    folder = Path("../../input_data")

    files = [f for f in folder.iterdir() if f.is_file()]

    if not files:
        print("Folder jest pusty.")
    else:
        files[0].unlink()
        print(f" Usunięto plik: {files[0].name}")



In [426]:
# CHecking if method above works
use_model("nn_exoplanets0")


FileNotFoundError:  Brak pliku CSV w folderze: ../../input_data